In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from collections import Counter
from sklearn.datasets import make_classification
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from keras.utils.vis_utils import plot_model
plt.switch_backend('agg')

Using Theano backend.


In [0]:
length = 1000
words = 20000
dim = 100

In [24]:
df = pd.read_excel('/content/drive/My Drive/dataset/mtsamples_v2_exel.xlsx')
df = df.dropna()
df = df.reset_index(drop=True)
print('Input Shape ',df.shape)
# print(df.columns)
print('No. of Classes',len(set(df['medical_specialty'])))
print(sorted(set(df['medical_specialty'])))
# Set it to None to display all columns in the dataframe
# pd.set_option('display.max_columns', None)
# print(empDfObj)

Input Shape  (3898, 6)
No. of Classes 39
[' Allergy / Immunology', ' Bariatrics', ' Cardiovascular / Pulmonary', ' Chiropractic', ' Consult - History and Phy.', ' Cosmetic / Plastic Surgery', ' Dentistry', ' Dermatology', ' Diets and Nutritions', ' Discharge Summary', ' ENT - Otolaryngology', ' Emergency Room Reports', ' Endocrinology', ' Gastroenterology', ' General Medicine', ' Hematology - Oncology', ' Hospice - Palliative Care', ' IME-QME-Work Comp etc.', ' Lab Medicine - Pathology', ' Letters', ' Nephrology', ' Neurology', ' Neurosurgery', ' Obstetrics / Gynecology', ' Office Notes', ' Ophthalmology', ' Orthopedic', ' Pain Management', ' Pediatrics - Neonatal', ' Physical Medicine - Rehab', ' Podiatry', ' Psychiatry / Psychology', ' Radiology', ' Rheumatology', ' SOAP / Chart / Progress Notes', ' Sleep Medicine', ' Speech - Language', ' Surgery', ' Urology']


In [25]:
num=sorted(set(df['medical_specialty']))
nid = dict((note, number) for number, note in enumerate(num))
def fnid(i):
    return nid[i]

df['medical_specialty']=df['medical_specialty'].apply(fnid)
print(sorted(set(df['medical_specialty'])))
# print(df['medical_specialty'].shape)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [0]:
texts = []
labels = []
def clean_str(string):
    
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\"", "", string)
    string = re.sub(r"\'", "", string)
    
    return string.strip().lower()

for index in range(df.keywords.shape[0]):
    text = BeautifulSoup(df.keywords[index])
    texts.append(clean_str(str(text.get_text().encode())))

for index in df['medical_specialty']:
    labels.append(index)

In [27]:
token = Tokenizer(num_words=length)
token.fit_on_texts(texts)
seq = token.texts_to_sequences(texts)
word_index = token.word_index
print('No. tokens: ',len(word_index))

No. tokens:  5928


In [28]:
data = pad_sequences(seq, maxlen=length)
print("Input: ", data.shape)
labels = to_categorical(np.asarray(labels))
indxs = np.arange(data.shape[0])
np.random.shuffle(indxs)
data = data[indxs]
labels = labels[indxs]
X_train, x_test, Y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=65)
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=69)
print("Training samples: ", x_train.shape)
print("Training labels: ", y_train.shape)
print("Testing samples: ", x_test.shape)
print("Testing labels: ", y_test.shape)
print("Validation samples: ", x_val.shape)
print("Validation labels: ", y_val.shape)

Input:  (3898, 1000)
Training samples:  (2494, 1000)
Training labels:  (2494, 39)
Testing samples:  (780, 1000)
Testing labels:  (780, 39)
Validation samples:  (624, 1000)
Validation labels:  (624, 39)


In [30]:
embeddings_index = {}
f = open("/content/drive/My Drive/dataset/glove.6B.100d.txt",encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [0]:
embedding_matrix = np.random.random((len(word_index) + 1, dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            dim,weights=[embedding_matrix],
                            input_length=length,trainable=True)

In [0]:
from keras import backend as K

def recall_m(orignal, predicted):
    true_positives = K.sum(K.round(K.clip(orignal * predicted, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(orignal, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [43]:
sequence_input = Input(shape=(length,), dtype='int32')

input = embedding_layer(sequence_input)
x = Conv1D(64, 5, activation='relu')(input)
x = Conv1D(64, 5, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.2)(x)

x = Conv1D(128, 5, activation='relu')(x)
x = Conv1D(128, 5, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.2)(x)

x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.2)(x)

x = Flatten()(x)

x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)

# l_dense1 = Dense(128, activation='relu')(l_dense)
preds = Dense(len(macronum), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc',f1_m,precision_m, recall_m])
model.save('research_CNN.model')
print("CNN")
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)
print(model.metrics_names)

CNN
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 100)         592900    
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 996, 64)           32064     
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 992, 64)           20544     
_________________________________________________________________
batch_normalization_16 (Batc (None, 992, 64)           256       
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 198, 64)           0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 198, 64)           

In [0]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=2,callbacks=[cp])

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Train on 2494 samples, validate on 624 samples
Epoch 1/10
2494/2494 [==============================] - 196s 78ms/step - loss: 3.0425 - acc: 0.2538 - f1_m: 0.0184 - precision_m: 0.0265 - recall_m: 0.0144 - val_loss: 10.9434 - val_acc: 0.2564 - val_f1_m: 0.2564 - val_precision_m: 0.2564 - val_recall_m: 0.2564

Epoch 00001: val_acc improved from -inf to 0.25641, saving model to model_cnn.hdf5
Epoch 2/10
 680/2494 [=======>......................] - ETA: 2:44 - loss: 2.9986 - acc: 0.2603 - f1_m: 0.0216 - precision_m: 0.0324 - recall_m: 0.0162

In [36]:
model.load_weights('/content/model_cnn.hdf5')
loss, acc, f1_m, precision_m, recall_m= model.evaluate(x_test,y_test, verbose=0)
print(model.metrics_names)
print("Loss: ", loss)
print("Accuracy: ", acc)
print("F1 score: ", f1_m)
print("Precision: ", precision_m)
print("Recall: ", recall_m)

NameError: ignored

In [0]:
print(tr_pr.shape)


In [0]:
fig3 = plt.figure()
plt.plot(history.tr_pr['loss'],'r')
plt.plot(history.tr_pr['val_loss'],'b')
plt.legend(['Training loss', 'Validation Loss'])
plt.xlabel('Epochs ')
plt.ylabel('Loss')
plt.title('Loss Curves :CNN')
fig3.savefig('loss_cnn.png')
plt.show(fig3)

AttributeError: ignored

In [0]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'b')
plt.legend(['Training loss', 'Validation Loss'])
plt.xlabel('Epochs ')
plt.ylabel('Loss')
plt.title('Loss Curves :CNN')
fig1.savefig('loss_cnn.png')
plt.show(fig1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Passing the block parameter of show() positionally is deprecated since Matplotlib 3.1; the parameter will become keyword-only in 3.3.
  if __name__ == '__main__':


In [0]:
fig2=plt.figure()
plt.plot(history.history['acc'],'r')
plt.plot(history.history['val_acc'],'b')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.xlabel('Epochs ')
plt.ylabel('Accuracy')
plt.title('Accuracy Curves : CNN')
fig2.savefig('accuracy_cnn.png')
plt.show()